In [ ]:
!pip install transformers
!pip install accelerate -U
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset
import pandas as pd
import numpy as np
from transformers import BertForSequenceClassification

In [ ]:
file_path = '/content/drive/MyDrive/Arnon/'
df_train = pd.read_csv(f'{file_path}train.csv')
df_test = pd.read_csv(f'{file_path}test.csv')

In [ ]:
df_train.shape
df_test.shape

(3080, 2)

In [ ]:
df_train.head()
df_test.head()

,text,category
0,How do I locate my card?,card_arrival
1,"I still have not received my new card, I order...",card_arrival
2,I ordered a card but it has not arrived. Help ...,card_arrival
3,Is there a way to know when my card will arrive?,card_arrival
4,My card has not arrived yet.,card_arrival


In [ ]:
df_train['category'].value_counts()

card_payment_fee_charged                            187
direct_debit_payment_not_recognised                 182
balance_not_updated_after_cheque_or_cash_deposit    181
wrong_amount_of_cash_received                       180
cash_withdrawal_charge                              177
                                                   ... 
lost_or_stolen_card                                  82
card_swallowed                                       61
card_acceptance                                      59
virtual_card_not_working                             41
contactless_not_working                              35
Name: category, Length: 77, dtype: int64

In [ ]:
possible_labels = df_train.category.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'card_arrival': 0,
 'card_linking': 1,
 'exchange_rate': 2,
 'card_payment_wrong_exchange_rate': 3,
 'extra_charge_on_statement': 4,
 'pending_cash_withdrawal': 5,
 'fiat_currency_support': 6,
 'card_delivery_estimate': 7,
 'automatic_top_up': 8,
 'card_not_working': 9,
 'exchange_via_app': 10,
 'lost_or_stolen_card': 11,
 'age_limit': 12,
 'pin_blocked': 13,
 'contactless_not_working': 14,
 'top_up_by_bank_transfer_charge': 15,
 'pending_top_up': 16,
 'cancel_transfer': 17,
 'top_up_limits': 18,
 'wrong_amount_of_cash_received': 19,
 'card_payment_fee_charged': 20,
 'transfer_not_received_by_recipient': 21,
 'supported_cards_and_currencies': 22,
 'getting_virtual_card': 23,
 'card_acceptance': 24,
 'top_up_reverted': 25,
 'balance_not_updated_after_cheque_or_cash_deposit': 26,
 'card_payment_not_recognised': 27,
 'edit_personal_details': 28,
 'why_verify_identity': 29,
 'unable_to_verify_identity': 30,
 'get_physical_card': 31,
 'visa_or_mastercard': 32,
 'topping_up_by_card': 33,
 '

In [ ]:
df_train['label'] = df_train.category.replace(label_dict)
df_test['label'] =  df_test.category.replace(label_dict)

In [ ]:
df_train
df_test

,text,category,label
0,How do I locate my card?,card_arrival,0
1,"I still have not received my new card, I order...",card_arrival,0
2,I ordered a card but it has not arrived. Help ...,card_arrival,0
3,Is there a way to know when my card will arrive?,card_arrival,0
4,My card has not arrived yet.,card_arrival,0
...,...,...,...
3075,"If i'm not in the UK, can I still get a card?",country_support,76
3076,How many countries do you support?,country_support,76
3077,What countries do you do business in?,country_support,76
3078,What are the countries you operate in.,country_support,76


In [ ]:
from sklearn.model_selection import train_test_split
#no need for splitting as we have data split for us
""" X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.label.values,
                                                  test_size=0.15,
                                                  random_state=42,
                                                  stratify=df.label.values)
                                                  """
X_train, y_train,  = df_train['text'], df_train['label']
X_val, y_val = df_test['text'], df_test['label']

In [ ]:
#df_test['data_type'] = ['not_set']*df.shape[0]
#df_train.loc[X_train]
#df.loc[X_train, 'data_type'] = 'train'
#df.loc[X_val, 'data_type'] = 'val'
""" no need for data typing if using different df"""

' no need for data typing if using different df'

In [ ]:
df_train.groupby(['category', 'label']).count()

,,text
category,label,
Refund_not_showing_up,44,162
activate_my_card,71,159
age_limit,12,110
apple_pay_or_google_pay,74,126
atm_support,36,87
...,...,...
virtual_card_not_working,63,41
visa_or_mastercard,32,135
why_verify_identity,29,121


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df_train.text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df_test.text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df_train.label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df_test.label.values)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)

(10003, 3080)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train,
                              sampler=RandomSampler(dataset_train),
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def accuracy_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return accuracy_score(labels_flat, preds_flat)

def precision_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    try:
        # Use zero_division=1 to set precision to 1.0 when there are no predicted samples for a class
        precision = precision_score(labels_flat, preds_flat, average='weighted', zero_division=1)
    except UndefinedMetricWarning as e:
        precision = 1.0  # Set precision to 1.0 when there are no predicted samples

    return precision

def recall_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return recall_score(labels_flat, preds_flat, average='weighted')

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def confusion_matrix_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return confusion_matrix(labels_flat, preds_flat)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):

    model.train()

    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


    torch.save(model.state_dict(), f'{file_path}finetuned_BERT_epoch_{epoch}.model')

    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/3335 [00:00<?, ?it/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
model.load_state_dict(torch.load(f'{file_path}finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)
val_f1 = f1_score_func(predictions, true_vals)
acc = accuracy_func(predictions, true_vals)
precision = precision_func(predictions, true_vals)
recall = recall_func(predictions, true_vals)
confusion = confusion_matrix_func(predictions, true_vals)

In [ ]:
print(f"f1 score: {val_f1}")
print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Confusion Metrix: {confusion}")

f1 score: 0.758355093764913
Accuracy: 0.7808441558441559
Precision: 0.789485741097893
Recall: 0.7808441558441559
Confusion Metrix: [[34  2  0 ...  0  0  0]
 [ 0 39  0 ...  0  0  0]
 [ 0  0 38 ...  0  0  0]
 ...
 [ 0  0  0 ... 40  0  0]
 [ 0  0  0 ...  0 22  0]
 [ 0  0  0 ...  0  0 37]]


In [ ]:
accuracy_per_class(predictions, true_vals)

Class: card_arrival
Accuracy: 34/40

Class: card_linking
Accuracy: 39/40

Class: exchange_rate
Accuracy: 38/40

Class: card_payment_wrong_exchange_rate
Accuracy: 38/40

Class: extra_charge_on_statement
Accuracy: 36/40

Class: pending_cash_withdrawal
Accuracy: 33/40

Class: fiat_currency_support
Accuracy: 30/40

Class: card_delivery_estimate
Accuracy: 21/40

Class: automatic_top_up
Accuracy: 35/40

Class: card_not_working
Accuracy: 37/40

Class: exchange_via_app
Accuracy: 24/40

Class: lost_or_stolen_card
Accuracy: 21/40

Class: age_limit
Accuracy: 40/40

Class: pin_blocked
Accuracy: 32/40

Class: contactless_not_working
Accuracy: 21/40

Class: top_up_by_bank_transfer_charge
Accuracy: 10/40

Class: pending_top_up
Accuracy: 38/40

Class: cancel_transfer
Accuracy: 36/40

Class: top_up_limits
Accuracy: 39/40

Class: wrong_amount_of_cash_received
Accuracy: 34/40

Class: card_payment_fee_charged
Accuracy: 37/40

Class: transfer_not_received_by_recipient
Accuracy: 30/40

Class: supported_card

In [ ]:
inputs = tokenizer("I lost my card", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)

tokenizer.decode(predicted_token_id)

RuntimeError: ignored